In [1]:
from bs4 import BeautifulSoup as bsoup
import urllib.robotparser
import requests
import time
import pandas as pd
import numpy as np
import re
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [5]:
url = 'https://www.boxofficemojo.com/date/2021-08-08/'
page = requests.get(url)
soup = bsoup(page.text, 'lxml')

all_divs = soup.main.find_all("div", id="table")
all_tables = all_divs[0].find_all("table")
all_trs = all_tables[0].find_all("tr")

for i in range(1, len(all_trs)):
    print(f"Entry row {i} = ")
    
    all_tds = all_trs[i].find_all("td")
    entry = []
    
    for j in range(0, len(all_tds)):
        entry.append(all_tds[j].text)       
    
    print(entry)

Entry row 1 = 
['1', '1', 'The Suicide Squad', '$5,940,000', '-27.6%', '-', '4,002', '$1,484', '$26,250,000', '3', 'Warner Bros.\n\n', 'false', 'false']
Entry row 2 = 
['2', '2', 'Jungle Cruise', '$4,616,034', '-30.8%', '-50%', '4,310', '$1,071', '$65,443,044', '10', 'Walt Disney Studios Motion Pictures\n\n', 'false', 'false']
Entry row 3 = 
['3', '3', 'Old', '$1,213,725', '-28.3%', '-41.4%', '3,138', '$386', '$38,498,750', '17', 'Universal Pictures\n\n', 'false', 'false']
Entry row 4 = 
['4', '4', 'Black Widow', '$1,082,915', '-35.1%', '-42.3%', '3,100', '$349', '$174,280,036', '31', 'Walt Disney Studios Motion Pictures\n\n', 'false', 'false']
Entry row 5 = 
['5', '5', 'Stillwater', '$823,955', '-32.9%', '-41.1%', '2,611', '$315', '$9,968,760', '10', 'Focus Features\n\n', 'false', 'false']
Entry row 6 = 
['6', '6', 'The Green Knight', '$770,000', '-23.4%', '-55.7%', '735', '$1,047', '$12,081,727', '10', 'A24\n\n', 'false', 'false']
Entry row 7 = 
['7', '7', 'Snake Eyes', '$522,121', '

In [14]:
# Build date for URL and make requests for each day

month_and_days = {'01': [x for x in range(1,32)], '02': [x for x in range(1,29)], '03':[x for x in range(1,32)],
                  '04':[x for x in range(1,31)],'05':[x for x in range(1,32)], '06':[x for x in range(1,31)], 
                 '07':[x for x in range(1,32)], '08':[x for x in range(1,32)], '09':[x for x in range(1,31)],
                 '10': [x for x in range(1,32)], '11':[x for x in range(1,31)], '12':[x for x in range(1,32)]}

leap_years = [2008, 2016, 2020]
leap_year_month_days = {'01': [x for x in range(1,32)], '02': [x for x in range(1,30)], '03':[x for x in range(1,32)],
                  '04':[x for x in range(1,31)],'05':[x for x in range(1,32)], '06':[x for x in range(1,31)], 
                 '07':[x for x in range(1,32)], '08':[x for x in range(1,32)], '09':[x for x in range(1,31)],
                 '10': [x for x in range(1,32)], '11':[x for x in range(1,31)], '12':[x for x in range(1,32)]}

all_data = []

for year in range(2015, 2021):
    # build url for each year/month/day
    url = 'https://www.boxofficemojo.com/date/'
    
    if year in leap_years:
        m_d = leap_year_month_days
    else:
        m_d = month_and_days
        
    y = str(year)
    
    for month in m_d:
        m = '-' + month
        for day in m_d[month]:
            if len(str(day)) == 1:
                d = '-' + '0' + str(day)
            else:
                d = '-' + str(day)
            
            date = y + m + d
            url_curr = url + date
            
            #get data
            page = requests.get(url_curr)
            soup = bsoup(page.text, 'lxml')
            
            all_divs = soup.main.find_all("div", id="table")
            try:
                all_tables = all_divs[0].find_all("table")
            except:
                print(date)
            
            try:
                all_trs = all_tables[0].find_all("tr")
            except:
                print(date)
            
            for i in range(1, len(all_trs)):

                all_tds = all_trs[i].find_all("td")
                entry = []

                for j in range(0, len(all_tds)):
                    entry.append(all_tds[j].text)       
                
                entry = entry[0:11]
                entry.append(date)

                all_data.append(entry)
    print(f'{year} processed...')

2015 processed...
2016 processed...
2017 processed...
2018 processed...
2019 processed...
2020-03-28
2020-03-29
2020-03-30
2020-03-31
2020-04-01
2020-04-02
2020 processed...


In [13]:
all_data[0]

['1',
 '1',
 'The Hobbit: The Battle of the Five Armies',
 '$9,320,601',
 '+55.3%',
 '-29%',
 '3,875',
 '$2,405',
 '$198,869,927',
 '16',
 'Warner Bros.\n\n',
 'false',
 '2015-01-01']

In [30]:
column_names = ['TD', 'YD', 'Movie', 'Daily', '%YD', '%LW', 'Theaters', 'Avg', 'To_Date', 'Days', 'Distributor', 'Date']
df = pd.DataFrame(all_data, columns=column_names)
df

,TD,YD,Movie,Daily,%YD,%LW,Theaters,Avg,To_Date,Days,Distributor,Date
0,1,1,The Hobbit: The Battle of the Five Armies,"$9,320,601",+55.3%,-29%,"3,875","$2,405","$198,869,927",16,Warner Bros.\n\n,2015-01-01
1,2,3,Into the Woods,"$8,081,110",+52.7%,-46.4%,"2,507","$3,223","$72,143,214",8,Walt Disney Studios Motion Pictures\n\n,2015-01-01
2,3,2,Unbroken,"$6,953,045",+17.4%,-55%,"3,167","$2,195","$69,481,655",8,Universal Pictures\n\n,2015-01-01
3,4,4,Night at the Museum: Secret of the Tomb,"$5,974,323",+22.5%,-16.4%,"3,914","$1,526","$75,328,905",14,Twentieth Century Fox\n\n,2015-01-01
4,5,5,Annie,"$4,257,057",+42%,-6.8%,"3,197","$1,331","$61,208,977",14,Sony Pictures Entertainment (SPE)\n\n,2015-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...
103750,13,13,All My Life,"$2,715",-22.9%,-61.6%,133,$20,"$899,585",28,Universal Pictures\n\n,2020-12-31
103751,14,14,Let Him Go,"$1,950",+66%,-80.5%,54,$36,"$9,347,940",56,Focus Features\n\n,2020-12-31
103752,15,15,The Forgotten Carols,"$1,002",+18.6%,-92.4%,48,$20,"$500,692",42,Purdie Distribution\n\n,2020-12-31
103753,16,16,The Rescue,$318,+31.4%,-81.6%,7,$45,"$18,490",14,CMC Pictures\n\n,2020-12-31


In [31]:
# Replace non-numeric values for numeric data columns
df['Daily'] = df['Daily'].str.replace('[^0-9]', '')
df['To_Date'] = df['To_Date'].str.replace('[^0-9]', '')
df['%YD'] = df['%YD'].str.replace('+', '')
df['%YD'] = df['%YD'].str.replace('-', '')
df['%YD'] = df['%YD'].str.replace('%', '')
df['%YD'] = df['%YD'].str.replace(',', '')
df['%YD'] = df['%YD'].str.replace('<0.1', '0')
df['%LW'] = df['%LW'].str.replace('+', '')
df['%LW'] = df['%LW'].str.replace('-', '')
df['%LW'] = df['%LW'].str.replace('%', '')
df['%LW'] = df['%LW'].str.replace(',', '')
df['%LW'] = df['%LW'].str.replace('<0.1', '0')
df['Theaters'] = df['Theaters'].str.replace('[^0-9]', '')
df['Avg'] = df['Avg'].str.replace('[^0-9]', '')
df['Days'] = df['Days'].str.replace('[^0-9]', '')
df['Distributor'] = df['Distributor'].str.replace('\\n', '')
df.head()

,TD,YD,Movie,Daily,%YD,%LW,Theaters,Avg,To_Date,Days,Distributor,Date
0,1,1,The Hobbit: The Battle of the Five Armies,9320601,55.3,29,3875,2405,198869927,16,Warner Bros.,2015-01-01
1,2,3,Into the Woods,8081110,52.7,46.4,2507,3223,72143214,8,Walt Disney Studios Motion Pictures,2015-01-01
2,3,2,Unbroken,6953045,17.4,55,3167,2195,69481655,8,Universal Pictures,2015-01-01
3,4,4,Night at the Museum: Secret of the Tomb,5974323,22.5,16.4,3914,1526,75328905,14,Twentieth Century Fox,2015-01-01
4,5,5,Annie,4257057,42,6.8,3197,1331,61208977,14,Sony Pictures Entertainment (SPE),2015-01-01


In [32]:
# Update numeric data types
df['Daily'] = pd.to_numeric(df['Daily'])
df['%YD'] = pd.to_numeric(df['%YD'])
df['%LW'] = pd.to_numeric(df['%LW'])
df['Theaters'] = pd.to_numeric(df['Theaters'])
df['Avg'] = pd.to_numeric(df['Avg'])
df['Days'] = pd.to_numeric(df['Days'])
df.dtypes

TD              object
YD              object
Movie           object
Daily            int64
%YD            float64
%LW            float64
Theaters       float64
Avg            float64
To_Date         object
Days           float64
Distributor     object
Date            object
dtype: object

In [33]:
df['Date'] = pd.to_datetime(df['Date'])
df.sample(20)

,TD,YD,Movie,Daily,%YD,%LW,Theaters,Avg,To_Date,Days,Distributor,Date
82901,29,30,The Best of Enemies,8711,49.6,61.3,134.0,65.0,10142851,33.0,STX Entertainment,2019-05-07
84760,21,18,The Souvenir,26497,18.6,48.9,145.0,182.0,740214,26.0,A24,2019-06-11
100186,5,4,Max Winslow and the House of Secrets,840,10.8,202.2,10.0,84.0,6108,17.0,3DLive AXO,2020-06-14
1131,12,12,Unbroken,986910,70.7,48.7,1606.0,614.0,111930235,31.0,Universal Pictures,2015-01-24
86414,37,35,Anna,27262,0.8,63.0,92.0,296.0,7468343,23.0,Lionsgate,2019-07-13
21654,31,32,Race,13826,352.1,1.9,90.0,153.0,19072676,57.0,Focus Features,2016-04-15
38489,60,55,1898: Our Last Men in the Philippines,191,74.6,NaN,10.0,19.0,2035,4.0,Sony Pictures Entertainment (SPE),2017-03-06
52959,32,33,Marshall,13171,45.0,67.5,111.0,118.0,8881134,45.0,Open Road Films (II),2017-11-26
77654,53,54,Replicas,1547,124.2,1.7,29.0,53.0,4041835,35.0,Entertainment Studios Motion Pictures,2019-02-14
13721,16,-,Truth,255000,NaN,NaN,1122.0,227.0,505577,15.0,Sony Pictures Classics,2015-10-30


In [34]:
df.dtypes

TD                     object
YD                     object
Movie                  object
Daily                   int64
%YD                   float64
%LW                   float64
Theaters              float64
Avg                   float64
To_Date                object
Days                  float64
Distributor            object
Date           datetime64[ns]
dtype: object

In [35]:
# Send data to pickle
df.to_pickle('clean-boxofficemojo-daily_gross.pkl')